<a href="https://colab.research.google.com/github/sinajahangir/Flood-Resilience-Analysis/blob/main/BayesianBeliefNetwrok_Flood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Belief Network for Flood Exposure Analysis

This notebook implements a Bayesian Belief Network (BBN) to model probabilistic relationships between flood exposure, population density, road network characteristics, and flood return periods for the Calgary road network.

## Overview

The BBN captures dependencies among:
- **Population Density Class** (`pop_d_c`): Low, Moderate, High
- **Return Period** (`rp`): Low (20-year), Moderate (100-year), High (500-year)
- **Road Class** (`rc`): Arterial, Collector, Expressway, Freeway, Local, Ramp, Resource
- **Flood Exposure Class** (`f_exp_c`): Low, Moderate, High

The network enables probabilistic inference to:
- Predict flood exposure probabilities given road characteristics and population density
- Infer road class distributions under high flood exposure conditions
- Analyze relationships between population density and road infrastructure types

## Author

**Mohammad Sina Jahangir**  
Email: mohammadsina.jahangir@gmail.com

Last updated: December 2025

## License

Copyright (c) [2024] [Mohammad Sina Jahangir]

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# Install and load libraries

In [ ]:
#Install BBN library
!pip install pgmpy

In [2]:
#Loading necessary libraries
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD as cpd
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from pgmpy.utils import get_example_model
import pandas as pd
import numpy as np
import networkx as nx
import pylab as plt
from pgmpy.estimators import ExpectationMaximization
from pgmpy.factors.discrete import TabularCPD

# Read data

In [ ]:
# Read the input csv
df=pd.read_csv('Exposure_All_BBFInput_v1.csv',index_col=0)

In [ ]:
# Assign values to return period classes
class_mapping = {
    500: 'High',
    100: 'Moderate',
    20: 'Low'
}
# Apply the mapping to the DataFrame
df = df.replace(class_mapping)

In [ ]:
# print variables
variables=list(df.columns)
print(variables)

In [ ]:
# print dataframe
print(df.head())

,pop_d,rp,pop_d_c,f_exp,f_exp_c,rc
0,3808.353808,Low,Moderate,0.000000,Low,Local
1,4353.348730,Low,Moderate,71.345166,Low,Local
2,2999.357739,Low,Moderate,184.895830,Moderate,Local
3,3952.702703,Low,Moderate,0.000000,Low,Local
4,2426.645519,Low,Low,2641.388520,Moderate,Ramp
...,...,...,...,...,...,...
5689,723.601178,High,Low,667.931539,Moderate,Local
5690,1398.074508,High,Low,0.000000,Low,Local
5691,679.284871,High,Low,426.818130,Low,Collector
5692,3296.406377,High,Moderate,196.879690,Low,Expressway


# Model development

In [ ]:
variables_mod=['pop_d_c','rp','rc','f_exp_c']
bayes_input = [(item, variables_mod[-1]) for item in variables_mod[:-1]]
bayes_input.append(('pop_d_c', 'rc'))

In [ ]:
model=DiscreteBayesianNetwork(bayes_input)

In [ ]:
# plot the model
fig=plt.figure(figsize=(10,10),dpi=100)
nx_graph = nx.DiGraph(model.edges())
nx.draw(nx_graph, with_labels=True)
plt.show()

In [ ]:
model.fit(df,estimator=ExpectationMaximization)

In [ ]:
# We have to change the probability of return periods
model.add_cpds(TabularCPD(variable='rp', variable_card=3, values=np.asarray([0.04,0.8,0.16]).reshape((-1,1)),state_names={'rp': ['High', 'Low', 'Moderate']}))

In [ ]:
print(model.get_cpds('rc'))

+----------------+-----+----------------------+
| pop_d_c        | ... | pop_d_c(Moderate)    |
+----------------+-----+----------------------+
| rc(Arterial)   | ... | 0.029292107404393815 |
+----------------+-----+----------------------+
| rc(Collector)  | ... | 0.21806346623270953  |
+----------------+-----+----------------------+
| rc(Expressway) | ... | 0.03742880390561432  |
+----------------+-----+----------------------+
| rc(Freeway)    | ... | 0.004882017900732303 |
+----------------+-----+----------------------+
| rc(Local)      | ... | 0.6883645240032547   |
+----------------+-----+----------------------+
| rc(Ramp)       | ... | 0.021969080553295363 |
+----------------+-----+----------------------+
| rc(Resource)   | ... | 0.0                  |
+----------------+-----+----------------------+


In [ ]:
print(model.get_cpds('pop_d_c'))

+-------------------+----------+
| pop_d_c(High)     | 0.050238 |
+-------------------+----------+
| pop_d_c(Low)      | 0.299841 |
+-------------------+----------+
| pop_d_c(Moderate) | 0.649921 |
+-------------------+----------+


In [ ]:
print(model.get_cpds('rp'))

+--------------+------+
| rp(High)     | 0.04 |
+--------------+------+
| rp(Low)      | 0.8  |
+--------------+------+
| rp(Moderate) | 0.16 |
+--------------+------+


In [ ]:
print(model.get_cpds('rp'))

+--------------+------+
| rp(High)     | 0.04 |
+--------------+------+
| rp(Low)      | 0.8  |
+--------------+------+
| rp(Moderate) | 0.16 |
+--------------+------+


# Inference

In [ ]:
infer = VariableElimination(model)  # Initialize infer object

In [ ]:
q = infer.query(variables=['rc'],evidence={'f_exp_c':'High'})
print(q)

+----------------+-----------+
| rc             |   phi(rc) |
+================+===========+
| rc(Arterial)   |    0.1255 |
+----------------+-----------+
| rc(Collector)  |    0.2939 |
+----------------+-----------+
| rc(Expressway) |    0.0556 |
+----------------+-----------+
| rc(Freeway)    |    0.0189 |
+----------------+-----------+
| rc(Local)      |    0.4337 |
+----------------+-----------+
| rc(Ramp)       |    0.0724 |
+----------------+-----------+
| rc(Resource)   |    0.0000 |
+----------------+-----------+


In [ ]:
q = infer.query(variables=['pop_d_c'],evidence={'f_exp_c':'High'})
print(q)

+-------------------+----------------+
| pop_d_c           |   phi(pop_d_c) |
+===================+================+
| pop_d_c(High)     |         0.1554 |
+-------------------+----------------+
| pop_d_c(Low)      |         0.2644 |
+-------------------+----------------+
| pop_d_c(Moderate) |         0.5802 |
+-------------------+----------------+


In [ ]:
q = infer.query(variables=['f_exp_c'],evidence={'rp':'High','pop_d_c':'High','rc':'Local'})
print(q)

+-------------------+----------------+
| f_exp_c           |   phi(f_exp_c) |
+===================+================+
| f_exp_c(High)     |         0.1235 |
+-------------------+----------------+
| f_exp_c(Low)      |         0.2716 |
+-------------------+----------------+
| f_exp_c(Moderate) |         0.6049 |
+-------------------+----------------+


In [ ]:
q = infer.query(variables=['f_exp_c'],evidence={'rp':'High','rc':'Expressway'})
print(q)

+-------------------+----------------+
| f_exp_c           |   phi(f_exp_c) |
+===================+================+
| f_exp_c(High)     |         0.0667 |
+-------------------+----------------+
| f_exp_c(Low)      |         0.1083 |
+-------------------+----------------+
| f_exp_c(Moderate) |         0.8250 |
+-------------------+----------------+


In [ ]:
q = infer.query(variables=['rc'],evidence={'pop_d_c':'High'})
print(q)

+----------------+-----------+
| rc             |   phi(rc) |
+================+===========+
| rc(Arterial)   |    0.0211 |
+----------------+-----------+
| rc(Collector)  |    0.1158 |
+----------------+-----------+
| rc(Expressway) |    0.0000 |
+----------------+-----------+
| rc(Freeway)    |    0.0105 |
+----------------+-----------+
| rc(Local)      |    0.8526 |
+----------------+-----------+
| rc(Ramp)       |    0.0000 |
+----------------+-----------+
| rc(Resource)   |    0.0000 |
+----------------+-----------+


In [ ]:
q = infer.query(variables=['rc'],evidence={'pop_d_c':'Low'})
print(q)

+----------------+-----------+
| rc             |   phi(rc) |
+================+===========+
| rc(Arterial)   |    0.1076 |
+----------------+-----------+
| rc(Collector)  |    0.3263 |
+----------------+-----------+
| rc(Expressway) |    0.1305 |
+----------------+-----------+
| rc(Freeway)    |    0.0229 |
+----------------+-----------+
| rc(Local)      |    0.3845 |
+----------------+-----------+
| rc(Ramp)       |    0.0229 |
+----------------+-----------+
| rc(Resource)   |    0.0053 |
+----------------+-----------+


# Save the model

In [ ]:
# Save the model
model.save('BBN_Model_RoadNetwork.bif', filetype='bif')  # BIF format